In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import torch, time, sys
import numpy as np
import scipy.integrate
solve_ivp = scipy.integrate.solve_ivp

In [ ]:
import os

cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in %r: %s" % (cwd, files))

In [ ]:
import pickle

def tel():
    dbfile = open('expert_rollout.pickle', 'rb')    
    expert_rollout = pickle.load(dbfile)
    dbfile.close()
    return np.array(expert_rollout[0].obs)
    
obs_expert_0 = tel()
#obs_expert_1 =np.transpose(np.array(expert_rollout[1].obs))

In [ ]:
#create energy ploy
def get_energy(x):
    T = 0.5 * 0.25 * x[:,4]**2 + 0.5 * (x[:,4]**2 + 0.25 * (x[:,4]+x[:,5])**2\
        + x[:,0]*(x[:,0]*x[:,1]-x[:,2]*x[:,3])*x[:,4]*(x[:,4]+x[:,5]) \
        + x[:,2]*(x[:,2]*x[:,1]+x[:,3]*x[:,0])*x[:,4]*(x[:,4]+x[:,5]) ) \
        + 0.5 * 1 * x[:,4]**2 + 0.5 * 1 * (x[:,4]+x[:,5])**2
    V = - 9.8 * x[:,0]/2 - 9.8 * (x[:,0] + 0.5*(x[:,0]*x[:,1]-x[:,2]*x[:,3]))
    return T+V
#import ipdb; ipdb.set_trace()
E_true_double = get_energy(obs_expert_0)
t_eval_double = np.arange(0 , np.shape(obs_expert_0)[0])
#import ipdb; ipdb.set_trace()

plt.subplot(2, 4, 4)
plt.plot(t_eval_double, E_true_double, 'k', label='Ground Truth', linewidth=2)

# plt.ylim(-1, 40)
plt.xlabel('t')
plt.title('Task 4: Acrobot')
plt.legend(fontsize=7)

In [ ]:
lw = 3 #linewidth
fs=9
ts=15
tpad = 7
ls=12

fig = plt.figure(figsize=[15,4], dpi=100)
plt.subplot(1,3,1)
plt.title('Trajectories', fontsize=ts, pad=tpad)
colors = ['orange', 'purple']

#Example trajectory
plt.plot(obs_expert_0[1], obs_expert_0[2], '-', c=colors[0], label='True path, roll out {}'.format(0), linewidth=2)
plt.plot(obs_expert_1[1], obs_expert_1[2], '--', c=colors[1], label='True path, roll out {}'.format(1), linewidth=2)
plt.axis('equal')
plt.xlabel('$x$', fontsize=ls) ; plt.ylabel('$y$', fontsize=ls)
plt.legend(fontsize=fs)

plt.subplot(1,3,2)
real_pe, real_ke, real_etot = potential_energy(orbit), kinetic_energy(orbit), total_energy(orbit)
plt.title('Ground truth energy', fontsize=ts, pad=tpad)
plt.xlabel('Time')
plt.plot(settings['t_eval'], real_pe, 'g:', label='Potential', linewidth=lw)
plt.plot(settings['t_eval'], real_ke, 'c-.', label='Kinetic', linewidth=lw)
plt.plot(settings['t_eval'], real_etot, 'k-', label='Total', linewidth=lw)
plt.legend(fontsize=fs)
plt.xlim(*settings['t_span'])
ymin = np.min([real_pe.min(), real_ke.min(), real_etot.min()])
ymax = np.max([real_pe.max(), real_ke.max(), real_etot.max()])
plt.ylim(ymin, ymax)

plt.subplot(1,3,3)
plt.title('Baseline NN energy', fontsize=ts, pad=tpad)
plt.xlabel('Time')
plt.plot(settings['t_eval'], potential_energy(base_orbit), 'g:', label='Potential', linewidth=lw)
plt.plot(settings['t_eval'], kinetic_energy(base_orbit), 'c-.', label='Kinetic', linewidth=lw)
plt.plot(settings['t_eval'], total_energy(base_orbit), 'k-', label='Total', linewidth=lw)
plt.legend(fontsize=fs)
plt.xlim(*settings['t_span'])
plt.ylim(ymin, ymax)
"""
plt.tight_layout() ; plt.show()
#fig.savefig('{}/orbits-base-example.{}'.format(args.fig_dir, FORMAT))